In [10]:
import pandas as pd
pd.options.mode.chained_assignment = None
from nltk.tokenize import word_tokenize


df = pd.read_csv('twitter_training.csv', header=None)


In [11]:
df.shape
# df.isnull()

(74682, 4)

In [12]:
df = df.drop([0,1], axis=1)

In [13]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df[2] = le.fit_transform(df[2])

df.head()

,2,3
0,3,im getting on borderlands and i will murder yo...
1,3,I am coming to the borders and I will kill you...
2,3,im getting on borderlands and i will kill you ...
3,3,im coming on borderlands and i will murder you...
4,3,im getting on borderlands 2 and i will murder ...


In [14]:
df.isnull().sum()

2      0
3    686
dtype: int64

In [15]:
df.dropna(inplace=True)

In [16]:
df.isnull().sum()

2    0
3    0
dtype: int64

In [17]:
df[2].value_counts()

2
1    22358
3    20655
2    18108
0    12875
Name: count, dtype: int64

In [18]:
sentencess = df[3]
type(sentencess)

pandas.core.series.Series

In [19]:
def lower(sent):
    return sent.lower()

In [20]:
import string
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

In [21]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
sentences = []

for i in sentencess:
    i = lower(i)
    
    #remove usernames
    i = re.sub(r"@(\w+)", "", i)
    
    #remove any urls
    i = re.sub(r"https?://\S+", "", i)
    
    #remove lagging spaces in b/w
    i = re.sub(r"\s{2,}", " ", i)
    
    #remove punctuations
    i = remove_punctuation(i)
    
    #stopwords
    stop_words = set(stopwords.words("english"))
    
    #split into words
    word_tokens = word_tokenize(i)
    
    #remove stopwords
    words = [word for word in word_tokens if word not in stop_words]
    
    #lemmatize words    
    res=[]
    for word in words:
       res.append(lemma.lemmatize(word,pos='v'))
    words=res
    
    #join thw words to form the final sentence
    i = " ".join(words)
    sentences.append(i)
    
    

In [22]:
sentences = pd.Series(sentences)
sentences

0                                im get borderlands murder
1                                         come border kill
2                                  im get borderlands kill
3                               im come borderlands murder
4                              im get borderlands 2 murder
                               ...                        
73991    realize windows partition mac like 6 years beh...
73992    realize mac window partition 6 years behind nv...
73993    realize windows partition mac 6 years behind n...
73994    realize windows partition mac like 6 years beh...
73995    like windows partition mac like 6 years behind...
Length: 73996, dtype: object

In [23]:
target = df[2]
target.shape

(73996,)

In [24]:
target

0        3
1        3
2        3
3        3
4        3
        ..
74677    3
74678    3
74679    3
74680    3
74681    3
Name: 2, Length: 73996, dtype: int32

In [25]:
sentences.to_list()
sentences.shape

(73996,)

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentences, target, test_size=0.20, random_state=0, stratify=target)

In [27]:
X_train

31845       want play league legends ping doesnt let annoy
68031    tesla truck isnt game ill disappoint elonmusk ...
41712    fm “ chicken dinner pubg ban india leave gamer...
22062    dear secret know cs go players huge technical ...
7516     come back overwatch 1 year break first comp ga...
                               ...                        
22594                                      keep great work
60744    alfa romeo celebrate 110 light years engine 54...
27916    beat titanfall 2 love think soon look happen b...
50271                                   retweet award shia
5578     hello amazon india happen action take order ca...
Length: 59196, dtype: object

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=2500)
tfidf_mat = vectorizer.fit_transform(X_train)

tfidf_mat = tfidf_mat.toarray()

tfidf_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
from sklearn.ensemble import RandomForestClassifier


classifierRF = RandomForestClassifier()


classifierRF.fit(tfidf_mat, y_train)

RandomForestClassifier()

In [30]:
# testing phase

test_mat = vectorizer.transform(X_test).toarray()

y_predRF = classifierRF.predict(test_mat)

In [31]:
from sklearn.metrics import accuracy_score, confusion_matrix

accRF = accuracy_score(y_test, y_predRF)

matrix = confusion_matrix(y_predRF, y_test)

print("Accuracy Randon Forest:  ",accRF)



print("Confusion Matrix\n",matrix)

Accuracy Randon Forest:   0.8688513513513514
Confusion Matrix
 [[2017   41   57   55]
 [ 148 4015  191  161]
 [ 115  132 3051  139]
 [ 295  284  323 3776]]


In [32]:
import pickle
pickle.dump(le, open("labelEncoder.pkl",'wb'))
pickle.dump(vectorizer, open("vectoriser.pkl",'wb'))
pickle.dump(classifierRF, open("classifier.pkl",'wb'))